In [1]:
import numpy as np
from PIL import Image
from scipy.ndimage import convolve
import cv2
from scipy import ndimage
from skimage.morphology import (erosion, dilation, closing, opening,
                                area_closing, area_opening)

def preprocess(image_name):
    # Tunable parameters. Can play around with these
    f = 11 # Boosting factor
    threshold = 25 # Threshold for binary image

    im = np.array(Image.open(image_name).convert("L"))

    # Sharpen image
    alpha = 0.05
    blurred = ndimage.gaussian_filter(im, 2)
    sharpened = im + alpha*(im-blurred)
    # new_img = Image.fromarray(sharpened.astype(np.uint8))
    # new_img.save("sharpened.jpg")

    # Opening operation
    element = np.array([[0,1,0],
                    [1,1,1],
                    [0,1,0]])
    im = opening(sharpened, element)

    # Boost filtering
    dx_filter = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], np.float32)
    dy_filter = np.array([[1, 2, 1], [0, 0, 0], [-1, -2, -1]], np.float32)

    Ix = convolve(im, dx_filter).astype(float)
    Iy = convolve(im, dy_filter).astype(float)
    sobel_im = (Ix*Ix + Iy*Iy)**0.5
    for i in range(sobel_im.shape[0]):
        for j in range(sobel_im.shape[1]):
            if sobel_im[i][j] > 255:
                sobel_im[i][j] = 255.
    
    sobel_im = sobel_im/255.

    g = im/255.
    z = np.zeros(im.shape)
    for i in range(z.shape[0]):
        for j in range(z.shape[1]):
            if sobel_im[i][j] > 0.5:
                z[i][j] = 0.4
            else:
                z[i][j] = 0.8
    w = np.zeros(im.shape)
    for i in range(w.shape[0]):
        for j in range(w.shape[1]):
            w[i][j] = (z[i][j]*(1 - sobel_im[i][j]) + (1-z[i][j])*(1-g[i][j])) * (f-8) + 8
    
    boosted_im = np.zeros(im.shape)
    for i in range(1, im.shape[0]-1):
        for j in range(1, im.shape[1]-1):
            high_boost_mask = 1/9*np.array([[-1, -1, -1], [-1, w[i][j], -1], [-1, -1, -1]], np.float32)
            temp = convolve(im[i-1:i+2, j-1:j+2], high_boost_mask)
            boosted_im[i][j] = temp[1][1]
    
    for i in range(1, im.shape[0]-1):
        for j in range(1, im.shape[1]-1):
            if boosted_im[i][j] > threshold:
                boosted_im[i][j] = 255
            else:
                boosted_im[i][j] = 0

    # new_img = Image.fromarray(boosted_im.astype(np.uint8))
    # new_img.save("boosted.jpg")
    return boosted_im

# Returns the neighbors of a given point
def neighbors(im, point):
    res = []
    if (point[0] < im.shape[0] - 1):
        res.append((point[0]+1, point[1]))
    if (point[0] > 1):
        res.append((point[0]-1, point[1]))
    if (point[1] < im.shape[1] - 1):
        res.append((point[0], point[1]+1))
    if (point[1] > 1):
        res.append((point[0], point[1]-1))
    return res

def search(labeled_im, cur_label, r, c):
    stack = []
    stack.append((r,c))
    while stack:
        cur_node = stack.pop()
        labeled_im[cur_node[0]][cur_node[1]] = cur_label
        cur_neighbors = neighbors(labeled_im, cur_node)
        for i,j in cur_neighbors:
            if (labeled_im[i][j] == -1):
                stack.append((i,j))

def find_components(labeled_im):
    cur_label = 0
    for i in range(labeled_im.shape[0]):
        for j in range(labeled_im.shape[1]):
            if labeled_im[i][j] == -1:
                cur_label += 1
                search(labeled_im, cur_label, i, j)
    print(str(cur_label) + " connected components")
    return labeled_im, cur_label

def label(im):
    labeled_im = np.zeros(im.shape)
    for i in range(labeled_im.shape[0]):
        for j in range(labeled_im.shape[1]):
            if im[i][j] == 0:
                labeled_im[i][j] = -1
            else:
                labeled_im[i][j] = 0
    
    return find_components(labeled_im)

import random
def label_illustration(labeled_im):
    label_to_color = {}
    r_channel = np.zeros(labeled_im.shape)
    g_channel = np.zeros(labeled_im.shape)
    b_channel = np.zeros(labeled_im.shape)
    ctr = 0
    for i in range(labeled_im.shape[0]):
        for j in range(labeled_im.shape[1]):
            if labeled_im[i][j] == 0:
                r = 255
                g = 255
                b = 255
            elif labeled_im[i][j] in label_to_color:
                r = label_to_color[labeled_im[i][j]][0]
                g = label_to_color[labeled_im[i][j]][1]
                b = label_to_color[labeled_im[i][j]][2]
            else:
                r = random.randint(0,255)
                g = random.randint(0,255)
                b = random.randint(0,255)
                # ctr += 1
                # if (ctr > 11):
                #     r = 255
                #     g = 255
                #     b = 255
                label_to_color[labeled_im[i][j]] = (r, g, b)
                
            r_channel[i][j] = r
            g_channel[i][j] = g
            b_channel[i][j] = b
    
    new_img = Image.fromarray(np.dstack((r_channel, g_channel, b_channel)).astype(np.uint8))
    new_img.save("labeled_image.jpg")

# im = preprocess('data/test_face.jpg')
# im = preprocess('data/leonardo.jpg')
# labeled_im, cur_label = label(im)
# label_illustration(labeled_im)

############ Calculating centre of mass and orientation and grouping ############
def calculate_mu(x_coords, y_coords, x_bar, y_bar, p, q):
    mu = 0
    for i in range(len(x_coords)):
        for j in range(len(y_coords)):
            mu += np.power((x_coords[i] - x_bar),p) * np.power((y_coords[j] - y_bar),q)
    return mu

def get_component_indices(labeled_im, cur_label):
    block_number_coords = np.zeros(cur_label)
    x_bar_all = np.zeros(cur_label)
    y_bar_all = np.zeros(cur_label)
    theta_all = np.zeros(cur_label)
    x_coords_all = []
    y_coords_all = []
    for cur_label in range(1, cur_label + 1):
        # print(cur_label)
        x_coords = np.where( labeled_im == cur_label )[0] 
        y_coords = np.where( labeled_im == cur_label )[1] 
        x_coords_all.append(x_coords)
        y_coords_all.append(y_coords)
        block_number_coords[cur_label - 1] = len(x_coords)

        x_bar_all[cur_label - 1] = int( np.average(x_coords) )
        y_bar_all[cur_label - 1] = int( np.average(y_coords) )

        if (len(x_coords)>5000 or len(y_coords>5000)):
            continue
        mu_1_1 = calculate_mu(x_coords, y_coords, x_bar_all[cur_label - 1], y_bar_all[cur_label - 1], 1, 1)
        mu_2_0 = calculate_mu(x_coords, y_coords, x_bar_all[cur_label - 1], y_bar_all[cur_label - 1], 2, 0)
        mu_0_2 = calculate_mu(x_coords, y_coords, x_bar_all[cur_label - 1], y_bar_all[cur_label - 1], 0, 2)

        if (mu_2_0 - mu_0_2) == 0: continue
        theta_all[cur_label - 1] = 0.5 * ( np.arctan2( 2*mu_1_1, (mu_2_0 - mu_0_2) ) )

    return x_bar_all, y_bar_all, theta_all, block_number_coords, x_coords_all, y_coords_all

# test_im = np.zeros((100,100))
# for i in range(45,55):
#     test_im[50][i] = 1.
#     test_im[51][i] = 1.
#     test_im[60][i] = 2.
#     test_im[61][i] = 2.
# x_bar_all, y_bar_all, theta_all, block_number_coords, x_coords_all, y_coords_all = get_component_indices(test_im, 2)
# print(x_bar_all)
# print(y_bar_all)
# print(theta_all)
# print(block_number_coords)
# print(x_coords_all)
# print(y_coords_all)
# print(len(np.unique(labeled_im)))
# print(cur_label)
# print(labeled_im.shape[0]*labeled_im.shape[1])
# x_bar_all, y_bar_all, theta_all, block_number_coords, x_coords_all, y_coords_all = get_component_indices(labeled_im, cur_label)

def find_blocks_within_radius(cur_block_idx, labeled_im, x_bar, y_bar, radius):
    min_y = int(np.floor(y_bar - radius))
    max_y = int(np.ceil(y_bar + radius))

    blocks_within_radius = []
    for y in range(min_y + 1, max_y):
        # From the formula: r^2 = (x-h)^2 + (y-k)^2, where 
        # h,k are the distances from x,y respectively
        x_diff = np.power(radius,2) - np.power(y-y_bar,2)
        min_x = int(np.floor(x_bar - x_diff))
        max_x = int(np.ceil(x_bar + x_diff))

        for x in range(min_x + 1, max_x):
            if x < labeled_im.shape[0] and y < labeled_im.shape[1] and labeled_im[x,y] != 0 and labeled_im[x,y] != cur_block_idx+1:
                if labeled_im[x,y] not in blocks_within_radius:
                    blocks_within_radius.append(labeled_im[x,y])
    return blocks_within_radius


def find_nearest_block(cur_block_idx, blocks_within_radius, x_bar, y_bar):
    min_distance_block = None
    min_distance = np.inf
    for block in blocks_within_radius:
        x_dist = np.power(x_bar[int(block - 1)] - x_bar[cur_block_idx], 2)
        y_dist = np.power(y_bar[int(block - 1)] - y_bar[cur_block_idx], 2)
        distance_to_cur_block = np.sqrt(x_dist + y_dist)
        if distance_to_cur_block < min_distance:
            min_distance = distance_to_cur_block
            min_distance_block = block

    return min_distance_block 

def grouping(labeled_im, cur_label, block_number_coords, x_bar, y_bar, x_coords_all, y_coords_all):
    N_max = 80 #??????? This changes based on the image - need to test different values
    for N in range(1, N_max + 1):
        radius = 8 - ( 6 * ( (N - 1) / (N_max - 1) ) )
        count = 0
        for i in range(cur_label):
            blocks_within_radius = []
            if block_number_coords[i] == N:
                blocks_within_radius = find_blocks_within_radius(i, labeled_im, x_bar[i], y_bar[i], radius)
            else:
                continue

            if len(blocks_within_radius) > 0:
                nearest_block = int(find_nearest_block(i, blocks_within_radius, x_bar, y_bar))
            else:
                continue

            #lies on same axis if any of the x coordinates are the same
            # lies_on_same_axis = bool(set(x_coords_all[nearest_block - 1]) & set(x_coords_all[i]))
            # lies_on_same_axis = False
            # if block_number_coords[nearest_block - 1] < N_max or lies_on_same_axis:
            block_to_merge_with = max(i + 1, nearest_block)
            block_to_merge = min(i + 1, nearest_block)

            # print(str(i+1) + " " + str(nearest_block))
            # print(str(block_to_merge_with) + " " + str(block_to_merge))

            x_coords = x_coords_all[block_to_merge - 1]
            y_coords = y_coords_all[block_to_merge - 1]

            labeled_im[x_coords, y_coords] = block_to_merge_with
            block_number_coords[block_to_merge_with-1] = block_number_coords[block_to_merge_with-1] + block_number_coords[block_to_merge-1]
            block_number_coords[block_to_merge-1] = 0
            new_x_coords = np.append(x_coords_all[block_to_merge_with-1], x_coords)
            new_y_coords = np.append(y_coords_all[block_to_merge_with-1], y_coords)
            x_coords_all[block_to_merge_with-1] = new_x_coords
            y_coords_all[block_to_merge_with-1] = new_y_coords
            x_coords_all[block_to_merge-1] = np.array([])
            y_coords_all[block_to_merge-1] = np.array([])

    return labeled_im

# Make labels consecutive in order after grouping
def relabel_grouped_im(labeled_im, num_labels):
    old_label_to_new_map = {}
    ctr = 0
    for i in range(labeled_im.shape[0]):
        for j in range(labeled_im.shape[1]):
            if labeled_im[i][j] == 0:
                continue
            if labeled_im[i][j] not in old_label_to_new_map:
                ctr += 1
                old_label_to_new_map[labeled_im[i][j]] = ctr
            labeled_im[i][j] = old_label_to_new_map[labeled_im[i][j]]
    if ctr != num_labels:
        print("something broke")
        print(ctr)
        print(num_labels)

# For testing
def use_only_good_labels(labeled_im, labels_to_use):
    num_labels = len(labels_to_use)
    old_label_to_new_map = {}
    ctr = 0
    for i in range(labeled_im.shape[0]):
        for j in range(labeled_im.shape[1]):
            if labeled_im[i][j] == 0:
                continue
            if labeled_im[i][j] not in labels_to_use:
                labeled_im[i][j] = 0
                continue
            if labeled_im[i][j] not in old_label_to_new_map:
                ctr += 1
                old_label_to_new_map[labeled_im[i][j]] = ctr
            labeled_im[i][j] = old_label_to_new_map[labeled_im[i][j]]
    if ctr != num_labels:
        print("something broke")
        print(ctr)
        print(num_labels)

            
# Default 60 length for testing
# Need to find a way to find length of semimajor axis
def get_block_lengths(x_bar_all, y_bar_all, theta_all, x_coords_all, y_coords_all):
    block_lengths = np.zeros(len(theta_all))
    for i in range(len(block_lengths)):
        block_lengths[i] = 60
    return block_lengths

In [38]:
import numpy as np
import math
class Block:
    def __init__(self,xcenter,ycenter,theta,length):
        #xcenter, ycenter: The center of our block
        #Theta: The orientation of the block
        #Length: The length of semimajor axis
        self.x = xcenter
        self.y = ycenter
        self.angle = theta
        self.length = length

    def GetCenter(self):
        return (self.x,self.y)

    def GetAngle(self):
        return self.angle

    def GetLength(self):
        return self.length

    def ComputeDistance(self,other):
        x2,y2 = other.GetCenter()
        return np.sqrt((self.x-x2)**2+(self.y-y2)**2)

def Baseline(block1,block2,threshold):
    x1,y1 = block1.GetCenter()
    x2,y2 = block2.GetCenter()
    a = y2-y1
    b = x1-x2
    c = x2 * y1 - x1 * y2
    theta = math.atan(-a/b)
    if theta < math.pi * -1/2 or theta >= math.pi/2:
        return []
    #print(a,b,c)
    D = block1.ComputeDistance(block2)
    l1 = block1.GetLength()/D
    l2 = block2.GetLength()/D
    ep =  (l1 - l2)**2
    ep += (l1 + l2-1)**2
    ep += (block1.GetAngle() - theta)**2
    ep += (block2.GetAngle() - theta)**2
    ep = -1.2 *ep
    score = math.exp(ep)
    if score < threshold:
        return []

    return [a,b,c,D,score]

def ComputeProbability(a,b,c,D,block,factor):
    x,y = block.GetCenter()
    d = np.abs(a*x+b*y+c)/np.sqrt(a**2+b**2)
    dlocal = factor * D
    ep = (d - dlocal)/D
    ep =-4.8 * (ep ** 2)
    return math.exp(ep)

def TotalProbability(reb,leb,mouth,nose,Weights=[0.5,0.2,0.1,0.1,0.1],Factors=[1,0.3,0.6],threshold=0.7):
    ret = Baseline(reb,leb,threshold)
    if len(ret) == 0:
        return 0
    
    a,b,c,D,score = ret
    score = score * Weights[0]
    score += Weights[1] * ComputeProbability(a,b,c,D,mouth,Factors[0])
    score += Weights[2] * ComputeProbability(a,b,c,D,reb,Factors[1])
    score += Weights[3] * ComputeProbability(a,b,c,D,leb,Factors[1])
    score += Weights[4] * ComputeProbability(a,b,c,D,nose,Factors[2])
    return score

def Matching(blocks):
    scores = []
    indices = []
    length = len(blocks)
    for reb in range(length):

        for leb in range(length):
            if reb == leb:
                continue

            for mouth in range(length):
                if mouth == reb or mouth == leb:
                    continue
                    
                for nose in range(length):
                    if nose == mouth or nose == reb or nose == leb:
                        continue
                    score = TotalProbability(blocks[reb],blocks[leb],blocks[mouth],blocks[nose]) 
                    if score > 0:
                        scores.append(score)
                        indices.append((reb,leb,mouth,nose))

    return (len(scores) != 0)

In [5]:
image_name = "data/test_face.jpg"
im = preprocess(image_name)
labeled_im, cur_label = label(im)
x_bar_all, y_bar_all, theta_all, block_number_coords, x_coords_all, y_coords_all = get_component_indices(labeled_im, cur_label)
labeled_im = grouping(labeled_im, cur_label, block_number_coords, x_bar_all, y_bar_all, x_coords_all, y_coords_all)
print(str(len(np.unique(labeled_im))) + " Connected Components")
num_labels = len(np.unique(labeled_im)) - 1
relabel_grouped_im(labeled_im, num_labels)

# good_labels = {1, 4, 6, 7, 10, 12, 14}
# 7 10 12 14
# use_only_good_labels(labeled_im, good_labels)

label_illustration(labeled_im)
x_bar_all, y_bar_all, theta_all, block_number_coords, x_coords_all, y_coords_all = get_component_indices(labeled_im, num_labels)
block_lengths = get_block_lengths(x_bar_all, y_bar_all, theta_all, x_coords_all, y_coords_all)


    # print(Matching(blocks))

1644 connected components
47 Connected Components


In [22]:
blocks = []
for i in range(len(block_lengths)):
    #print(x_bar_all[i],y_bar_all[i],theta_all[i],block_lengths[i])
    blocks.append(Block(x_bar_all[i], y_bar_all[i], theta_all[i], block_lengths[i]))

251.0 227.0 0.0 60.0
131.0 76.0 0.0 60.0
183.0 290.0 0.0 60.0
163.0 63.0 0.0 60.0
175.0 350.0 0.0 60.0
178.0 328.0 0.0 60.0
209.0 47.0 0.0 60.0
226.0 384.0 0.0 60.0
189.0 182.0 0.0 60.0
203.0 174.0 0.0 60.0
195.0 131.0 0.0 60.0
217.0 174.0 0.0 60.0
203.0 289.0 0.0 60.0
255.0 60.0 0.0 60.0
210.0 301.0 0.0 60.0
237.0 327.0 0.0 60.0
223.0 305.0 0.0 60.0
231.0 169.0 0.0 60.0
254.0 34.0 0.0 60.0
244.0 46.0 0.0 60.0
271.0 393.0 0.0 60.0
287.0 39.0 0.0 60.0
277.0 25.0 0.0 60.0
289.0 57.0 0.0 60.0
292.0 270.0 0.0 60.0
314.0 435.0 0.0 60.0
304.0 21.0 0.0 60.0
306.0 45.0 0.0 60.0
318.0 391.0 0.0 60.0
305.0 227.0 0.0 60.0
303.0 256.0 0.0 60.0
343.0 30.0 0.0 60.0
329.0 12.0 0.0 60.0
364.0 77.0 0.0 60.0
341.0 249.0 0.0 60.0
397.0 378.0 0.0 60.0
346.0 13.0 0.0 60.0
390.0 436.0 0.0 60.0
371.0 242.0 0.0 60.0
386.0 37.0 0.0 60.0
413.0 85.0 0.0 60.0
393.0 173.0 0.0 60.0
405.0 191.0 0.0 60.0
423.0 219.0 0.0 60.0
433.0 264.0 0.0 60.0
481.0 435.0 0.0 60.0


In [39]:
Matching(blocks)

KeyboardInterrupt: 